In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
  data = yf.download(ticker,start=start_date, end=end_date)
  df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker','Date'])
print(df.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-05-22  173.979996  174.710007  173.449997  174.199997  173.965073   
       2023-05-23  173.130005  173.380005  171.279999  171.559998  171.328644   
       2023-05-24  171.089996  172.419998  170.520004  171.839996  171.608261   
       2023-05-25  172.410004  173.899994  171.690002  172.990005  172.756714   
       2023-05-26  173.320007  175.770004  173.110001  175.429993  175.193420   

                     Volume  
Ticker Date                  
AAPL   2023-05-22  43570900  
       2023-05-23  50747300  
       2023-05-24  45143500  
      

In [6]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-05-22  173.979996  174.710007  173.449997  174.199997   
1   AAPL 2023-05-23  173.130005  173.380005  171.279999  171.559998   
2   AAPL 2023-05-24  171.089996  172.419998  170.520004  171.839996   
3   AAPL 2023-05-25  172.410004  173.899994  171.690002  172.990005   
4   AAPL 2023-05-26  173.320007  175.770004  173.110001  175.429993   

    Adj Close    Volume  
0  173.965073  43570900  
1  171.328644  50747300  
2  171.608261  45143500  
3  172.756714  56058300  
4  175.193420  54835000  


In [7]:
import plotly.express as px
fig = px.line(df, x='Date',
              y='Close',
              color='Ticker',
              title='Stock Market Performance for the last 3 months')
fig.show()

In [8]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price','Ticker':'Company'},
              title='Stock Price for Apple, Microsoft, Netflix and Google')
fig.show()

In [12]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
57  183.341002  188.684001
58  181.525002  187.870001
59  179.924002  186.943501
60  178.207002  185.987001
61  177.457002  185.114501

[62 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
186         NaN         NaN
187         NaN         NaN
188         NaN         NaN
189         NaN         NaN
190         NaN         NaN
..          ...         ...
243  130.398000  128.049999
244  130.236000  128.359499
245  130.283000  128.675999
246  130.452000  129.222500
247  130.409001  129.612500

[62 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
62          NaN         NaN
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
..          ...         ...
119  326.464999  335.43

In [13]:
for ticker, group in df.groupby('Ticker'):
  fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                title=f"{ticker} Moving Averages")
  fig.show()

In [15]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [17]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                 trendline='ols',
                 title='Correlation between Apple and Microsoft')
fig.show()